# Домашнее задание : ML как http-сервис

## Задача 1: применяем PCA-трансформацию

Модифицируйте файл `train.py` - добавьте в пайплайн обучения модели сжатие размерности до `n_components=2` с помощью [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) и обучите модель **в докере** на "сжатых" данных. Сохраните полученный объект `pca_transformer.pkl`, который умеет выполнять сжатие данных.

Решением домашки считается модифицированный файл *train.py*

In [ ]:
#@title Текст заголовка по умолчанию
# ---- train.py ----
import pickle
import os
import logging
from pathlib import Path

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

# логирование
LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)


# загрузка данных
def load_data(from_file: Path):
    data_source = np.genfromtxt(from_file.resolve().as_posix(), delimiter=',', skip_header=1)
    X = data_source[:, :3]
    y = data_source[:, 3]
    return X, y


# PCA
def transform_data(X, path: Path):
    pca_transformer = PCA(n_components=2).fit(X)
    data_pca = pca_transformer.transform(X)
    
    with path.open('wb') as f:
        pickle.dump(pca_transformer, f)
        logging.info(f'Модель сжата с помощью PCA и сохранена в {path.resolve()}' )

    return data_pca


# обучение модели
def train_model(X, y, path: Path):
    clf = DecisionTreeClassifier(max_depth=3, random_state=42)
    clf.fit(X, y)
    
    # сохраняем модель внутри контейнера в директории /www/classifier
    with open('clf.pkl', 'wb') as f:
        pickle.dump(clf, f)
        logging.info('Модель обучена и сохранена в %s' % Path().absolute())
        
def pipeline():
    X, y = load_data(Path('data/client_segmentation.csv'))
    X = transform_data(X, path=Path('./data/pca_transformer.pkl'))

    train_model(X, y, path=Path('./data/clf.pkl'))

pipeline()

## Задача 2: трансформация входных фичей на лету

Модифицируйте файл `service.py`: добавьте загрузку объекта для трансформации `pca_tansformer.pkl` и применяйте её **в докере** для трансформации набора входных фич в сжатые (см. метод `.transform()`):
<pre>
[x1, x2, x3] -> [x1_pca, x2_pca]
</pre>

Соответственно, predict надо выполнять на *сжатых* фичах


Решением домашки считается модифицированный файл *service.py*

In [ ]:
import json
import http.server
import logging
import os
import pickle
import socketserver
import sys
from http import HTTPStatus
from re import compile
from pathlib import Path

import numpy as np
from sklearn.tree import DecisionTreeClassifier

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)


def parse_params(params) -> dict:
    """
        Получаем параметры из GET-запроса
    """
    params_list = params.split('&')
    params_dict = {'x1': None, 'x2': None, 'x3': None}
    for param in params_list:
        key, value = param.split('=')
        params_dict[key] = float(value)
    return params_dict


class Handler(http.server.SimpleHTTPRequestHandler):
    """Простой http-сервер"""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_response(self) -> dict:
        """Пример запроса
        
        http://0.0.0.0:5000/classifier/?x1=1&x2=-2.2&x3=1.05
        """
        response = {'ping': 'ok'}
        params_parsed = self.path.split('?')
        
        if len(params_parsed) == 2 and self.path.startswith('/classifier'):
            params = params_parsed[1]
            params_dict = parse_params(params)
            response = params_dict
            
            user_features = np.array(
                [[params_dict['x1'],
                 params_dict['x2'], 
                 params_dict['x3']]]
            )
            
            #применим pca_transform
            reduced_features = transformer.transform(user_features)
            
            predicted_class = int(classifier_model.predict(reduced_features)[0])
            
            logging.info('predicted_class %s' % predicted_class)
            response.update({'predicted_class': predicted_class})
        
        elif self.path.startswith('/ping/'):
            response = {'message': 'pong'}

        return response

    def do_GET(self):
        # заголовки ответа
        self.send_response(HTTPStatus.OK)
        self.send_header("Content-type", "application/json")
        self.end_headers()
        self.wfile.write(json.dumps(self.get_response()).encode())


logging.info('Загружаем обученную модель')
with open('./data/clf.pkl', 'rb') as f:
    classifier_model = pickle.load(f)
    logging.info('Модель загружена: %s' % classifier_model)

with open('./data/pca_transformer.pkl', 'rb') as f:
    transformer = pickle.load(f)
    logging.info('PCA transformer загружен: %s' % transformer)

# Домашнее задание: рекомендательная система

## Задача 1: строим KNN

В реальной жизни KNN-рекомендатель не стоит делать на основе `sklearn.neighbors.NearestNeighbors` - есть готовые реализации, заточенные специально для построения рекомендательных систем. Хорошим примером такой реализации является [пакет implictit](). В рамках домашней работы предлагается разобраться с реализацией KNN-рекомендателя из этой библиотеки 

Почитайте документацию по модулю `implicit.nearest_neighbours.CosineRecommender`. Обучите KNN-рекомендатель и воспользуйтесь методом `recommend` для построения рекомендаций


In [4]:
import pandas as pd
import numpy as np
import implicit

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

from scipy.sparse import csr_matrix

In [6]:
content_views = pd.read_csv(
    'content_views.zip', delimiter=',', header=0, compression='zip',
    names = ['user_id', 'content_id', 'view_duration', 'view_ts', 'dt', 'platform'],
    dtype = {'user_id': np.uint32, 'content_id': np.uint16, 'view_duration': np.uint16},
    parse_dates = [3, 4]
)

print('Количество просмотров %s' % content_views.user_id.count())

content_views.head(3)

Количество просмотров 489565


,user_id,content_id,view_duration,view_ts,dt,platform
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG


In [7]:
content_description = pd.read_csv(
    'content_description.zip', delimiter=',', header=0, compression='zip',
    names = ['content_id', 'origin_country', 'release_date', 'kinopoisk_rating', 'compilation_id', 'genre'],
    dtype = {'content_id': np.uint16},
    parse_dates = [2]
)

print('Количество доступного контента %s' % content_description.content_id.count())

content_description.head(3)

Количество доступного контента 126182


,content_id,origin_country,release_date,kinopoisk_rating,compilation_id,genre
0,1974,87.0,2009-12-15,7.27,153,Для детей
1,2148,87.0,2009-12-21,7.27,153,Для детей
2,2184,87.0,2009-12-22,7.27,153,Для детей


In [8]:
user_encoder = LabelEncoder()
user_encoder.fit(content_views.user_id)

# переиндексация контента
content_views = content_views.assign(
    user_index = user_encoder.transform(content_views.user_id)
)

# кодируем индексы контента
item_encoder = LabelEncoder()
item_encoder.fit(content_views.content_id)

# новая переиндексация
content_views = content_views.assign(
    item_index = item_encoder.transform(content_views.content_id)
)


content_views.head()


,user_id,content_id,view_duration,view_ts,dt,platform,user_index,item_index
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG,802,22812
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG,2,20399
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG,911,19628
3,4498,30191,297,2019-03-18 15:27:18+03:00,2019-03-18,LG,773,13517
4,4886,39349,302,2019-03-18 12:08:16+03:00,2019-03-18,LG,836,16959


In [9]:
num_users = content_views.user_index.max() + 1
num_items = content_views.item_index.max() + 1
num_interactions = content_views.shape[0]

user_item = csr_matrix(
    (np.ones(num_interactions),(content_views.user_index.values, content_views.item_index.values)),
    shape=(num_users, num_items)
)
print('sparsity: %.4f' % (num_interactions / (num_users * num_items)))

user_item

sparsity: 0.0091


<2000x27012 sparse matrix of type '<class 'numpy.float64'>'
	with 259994 stored elements in Compressed Sparse Row format>

In [10]:
train_ids, test_ids = train_test_split(
    np.arange(start=0, stop=user_item.shape[0], step=1, dtype=np.uint32),
    test_size=0.2
)
print(
    "Размер обучающей выборки %d пользователей, размер валидационной выборки %d пользователей"
    % (train_ids.size, test_ids.size)
)

Размер обучающей выборки 1600 пользователей, размер валидационной выборки 400 пользователей


In [11]:
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares

random_user_index = 63

cosine_recommender = CosineRecommender(K=5, num_threads=0)
cosine_recommender.fit(user_item[train_ids,:])

cosine_recommender.recommend(random_user_index, user_item[train_ids,:])

[(1423, 1.0),
 (28, 0.9999999999999998),
 (1472, 0.7071067811865475),
 (151, 0.02011294966772824)]

## Задача 2: Item to Item

Решите задачу c2c рекомендаций - вызовите метод `similar_items` для  *item_id=1*

In [12]:
cosine_recommender.similar_items(1)

[(1, 1.0000000000000002),
 (745, 0.816496580927726),
 (1169, 0.07742096611387639),
 (218, 0.051502620262460476),
 (78, 0.04125904453875132)]

## Задача 3: обучаем Implicit

Почитайте документацию по модулю implicit.als.AlternatingLeastSquares. Обучите ALS-рекомендатель и воспользуйтесь методом recommend для построения рекомендаций

In [13]:
from implicit.als import AlternatingLeastSquares

als_recommender = AlternatingLeastSquares()
als_recommender.fit(user_item[train_ids,:])

als_recommender.recommend(random_user_index, user_item[train_ids,:])

GPU training requires factor size to be a multiple of 32. Increasing factors from 100 to 128.


[(814, 0.0038012224),
 (493, 0.0036843405),
 (1550, 0.003668075),
 (177, 0.0036490269),
 (631, 0.003647986),
 (279, 0.003632795),
 (863, 0.003628347),
 (887, 0.0036266975),
 (151, 0.0035908245),
 (776, 0.0035820673)]

## Задача 4: реализация метрик

Даны два вектора - истинная история пользователя и объекты, которые считает релеватными ваша модель

Вычислите

* precision
* recall
* precision@5


In [14]:
import numpy as np

user_interactions = [47315, 30004, 36322,  8942, 30820,  6086,  9126,   332, 16289,
       39106, 39335, 48506, 48654,  9234, 29935,  2678, 36202, 22636, 18007, 39328, 15414, 30016, 35601,
    58409, 21313,   386, 16303, 4397, 19644, 51887, 21659, 36325, 53030,  7764, 50266, 58734, 53419, 24121,
    50806, 36092,  8868, 28037, 36131, 13561, 16298, 27508, 41722, 30189, 46490,  2676, 43328, 781, 48397,
    41369, 39324, 36381, 39635, 27710, 47837, 28525, 12024, 56604, 41664, 37387, 48507, 413, 33526, 20059,
    49781, 56648, 16283, 50805, 34254, 39325, 59374, 22620,  8865, 27512, 13875, 30011,  7621,
    10544, 28076, 29716, 30054, 20490, 29466, 16852, 39363, 34250, 7024, 33541,   263, 21267, 25690, 23020,
    41368, 53414,  2681, 30201] 

user_recs = [
    50820, 27781, 36131, 50812, 36092, 12024, 59155, 30042, 15414, 19882, 21659, 27849, 39328, 34240, 2681,
    21267, 50126, 58560, 7764, 49781
]

def ap_(n, crossing_set, user_intercations):
    result = 0
    for x in np.nditer(crossing_set):
        result += 1/(user_interactions.index(x)+1)
    return result/n

doc = np.intersect1d(user_recs, user_interactions)

precision = len(doc)/len(user_recs)
recall = len(doc)/len(user_interactions)
avrpr_5 = ap_(5, doc, user_interactions)

print(f'Precision: {precision}\nRecall: {recall}\nPrecision@5: {avrpr_5}')

Precision: 0.5
Recall: 0.1
Precision@5: 0.05183403900280743
